## Flux comparions
This notebook compares the predictions of the R. opacus PD630 GSM with the fluxes from 13C-metabolic flux analysis (13C-MFA)

### Methods: 
<ol>
<li></li>
<li></li>
</ol>

### Imports

In [39]:
%matplotlib inline
from matplotlib import pyplot as plt
import cobra
import pandas as pd
import numpy as np
from edd_utils import login, export_study, export_metadata

### Load model 

In [2]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml")
model

Name,ropacus_annotated_curated
Memory address,0x07f2cbb3ca5d0
Number of metabolites,1583
Number of reactions,2385
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


In [3]:
# edd_study_slug = 'input-test-study'
# edd_server = 'public-edd.agilebiofoundry.org'
# user= 'garrettroell'
# session = login(edd_server=edd_server, user=user)
# # df = utils.load_study(edd_study_slug=edd_study_slug,edd_server=edd_server,
# #   default_to_system_user=False)

In [4]:
# df = export_study(session, edd_study_slug, edd_server=edd_server)
# df

### Get 13C MFA measured fluxes for phenol

In [40]:
glucose_fluxes = pd.read_csv('../13C_flux_data/13C_glucose_flux_data_2.csv')
test_df = glucose_fluxes[(glucose_fluxes['Pathway'] == 'Glucose Uptake') | (glucose_fluxes['Pathway'] == 'Glycolysis')]
test_df

,Pathway,Forward Reactions,Reverse Reactions,Reversed?,Reaction,Flux,90% Confidence Lower Bound,90% Confidence Upper Bound
0,Glucose Uptake,GLCpts,EX_glc__D_e_reverse_af641,NaN,Gluc.ext + ATP -> G6P,100.00,100.00,100.00
1,Glycolysis,PGI,PPGKr or HEX1_reverse_25efa,False,G6P <-> F6P,-1.61,-2.09,1.42
2,Glycolysis,F6PA and DHAK,FBP_reverse_bf2c9 or PFK,NaN,F6P + ATP -> FBP,0.00,0.00,1.91
3,Glycolysis,F6PA and DHAK,FBA_reverse_84806,False,FBP <-> DHAP + GAP,0.00,0.00,1.91
4,Glycolysis,TPI,TPI_reverse_c2c3b,False,DHAP <-> GAP,0.00,0.00,1.91
5,Glycolysis,GAPD and -1*PGK,GAPD_reverse_459c1 and PGK,NaN,GAP <-> 3PG + ATP + NADH,86.56,82.61,88.84
6,Glycolysis,( -1*PGM or PGM_1 ) and ENO,( PGM or PGM_1_reverse_02963 ) and ENO_reverse...,NaN,3PG <-> PEP,78.52,71.65,81.26
7,Glycolysis,-1*PPS or FRUpts2pp or SBTpts or MNLpts or ACG...,FRUpts2pp_reverse_55dac or SBTpts_reverse_74ed...,NaN,PEP <-> Pyr + ATP,19.02,-10.89,75.15


### Run glucose flux FBA

In [55]:
with model:
    model.objective = 'Growth_Glucose'
    medium = model.medium
    medium = {key:1000 for (key,value) in model.medium.items()}
    medium["EX_glc__D_e"] = 100.0
    medium["EX_phenol_e"] = 0.0
    model.medium = medium
    solution = model.optimize()
    
print(solution)

<Solution 11.824 at 0x7f2d6fb9b610>


Define a function to get fluxes and adjust for if flux value should have sign change

In [63]:
def get_flux_value(reaction_id, solution):
    if reaction_id.startswith('-1*'):
        reaction_id = reaction_id.split('-1*')[1]
        return -1*solution.fluxes[reaction_id]
    else:
        return solution.fluxes[reaction_id]

### Add Glucose FBA flux values to test_df 

###  Method
For each reaction in the metabolic flux analysis find the corresponding genome scale flux. Or relations = parallel reactions. And reactions = sequential reactions.
<ol>
    <li>Split 'or' relations that can be summed together</li>
    <li>For each 'and' relation find the minimum of the reactions and add that to the reaction flux</li>
    <li>Append the summed flux value to the FBA_flux list</li>
</ol>

In [65]:
FBA_fluxes = []
for _, row in test_df.iterrows():
    reactions = row['Forward Reactions']
    flux_value = 0
    for x in [x.strip('() ') for x in reactions.split(' or ')]:
        and_split = [y.strip('() ') for y in x.split(' and ')]
#         print(and_split)
#         fluxes = [get_flux_value(v, solution) for v in and_split]
#         min_flux = min(fluxes)
#         print(flux)
#         print('min_flux:', min_flux)
        flux_value += min([get_flux_value(v, solution) for v in and_split])
    FBA_fluxes.append(flux_value)
    
FBA_fluxes

# test_df['FBA Value'] = FBA_fluxes

# test_df
        
        
# temp.add(frozenset(y.strip('() ') for y in x.split(' and ')))


#     for y in or_split:
#         print([y.strip('() ') for y in or_split.split(' and ')])
    
#     print(row['Forward Reactions'])
#     print(solution.fluxes[reaction_id])

[100.0,
 96.74776287517679,
 51.793544395534326,
 51.793544395534326,
 73.55886468395364,
 152.4643564024877,
 138.98979192547355,
 131.8661196405389]

In [66]:
test_df['FBA Value'] = FBA_fluxes
test_df

/usr/local/share/jupyteruser/.pyenv/versions/biod_3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Pathway,Forward Reactions,Reverse Reactions,Reversed?,Reaction,Flux,90% Confidence Lower Bound,90% Confidence Upper Bound,FBA Value
0,Glucose Uptake,GLCpts,EX_glc__D_e_reverse_af641,NaN,Gluc.ext + ATP -> G6P,100.00,100.00,100.00,100.000000
1,Glycolysis,PGI,PPGKr or HEX1_reverse_25efa,False,G6P <-> F6P,-1.61,-2.09,1.42,96.747763
2,Glycolysis,F6PA and DHAK,FBP_reverse_bf2c9 or PFK,NaN,F6P + ATP -> FBP,0.00,0.00,1.91,51.793544
3,Glycolysis,F6PA and DHAK,FBA_reverse_84806,False,FBP <-> DHAP + GAP,0.00,0.00,1.91,51.793544
4,Glycolysis,TPI,TPI_reverse_c2c3b,False,DHAP <-> GAP,0.00,0.00,1.91,73.558865
5,Glycolysis,GAPD and -1*PGK,GAPD_reverse_459c1 and PGK,NaN,GAP <-> 3PG + ATP + NADH,86.56,82.61,88.84,152.464356
6,Glycolysis,( -1*PGM or PGM_1 ) and ENO,( PGM or PGM_1_reverse_02963 ) and ENO_reverse...,NaN,3PG <-> PEP,78.52,71.65,81.26,138.989792
7,Glycolysis,-1*PPS or FRUpts2pp or SBTpts or MNLpts or ACG...,FRUpts2pp_reverse_55dac or SBTpts_reverse_74ed...,NaN,PEP <-> Pyr + ATP,19.02,-10.89,75.15,131.866120


### Get 13C MFA measured fluxes

In [ ]:
glucose_flux_path = "../13C_flux_data/13C_glucose_flux_data.csv"
measured_glucose_df = pd.read_csv(glucose_flux_path)
measured_glucose_df

Eliminate all rows that do not have a defined reactionID

In [ ]:
measured_glucose_df = measured_glucose_df[~pd.isnull(measured_glucose_df['Reaction ID'])]
measured_glucose_df

In [ ]:
measured_flux_dict = {}
measured_reaction_ids = list(measured_glucose_df['Reaction ID'])
for reaction_id in measured_reaction_ids:
#     print(measured_glucose_df[measured_glucose_df['Reaction ID'] == reaction_id]['Reversed?'].values[0])
    measured_flux = measured_glucose_df[measured_glucose_df['Reaction ID'] == reaction_id]['Flux'].values[0] / 10
#     if measured_glucose_df[measured_glucose_df['Reaction ID'] == reaction_id]['Reversed?'].values[0]:
#         measured_flux = -1 * measured_flux
    measured_flux_dict[reaction_id] = measured_flux
measured_flux_dict

### Get FBA simulated fluxes

In [ ]:
simulated_flux_dict = {}
measured_reaction_ids = list(measured_glucose_df['Reaction ID'])
for reaction_id in measured_reaction_ids:
    simulated_flux_dict[reaction_id] = solution.fluxes[reaction_id]

simulated_flux_dict

### Plot FBA vs measured fluxes

In [ ]:
measured_flux_list = list(measured_flux_dict.values())
simulated_flux_list = list(simulated_flux_dict.values())
labels = list(measured_flux_dict.keys())

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(measured_flux_list, simulated_flux_list)
# ax.plot(ax.get_xlim(), ax.get_ylim(), ls="--", c=".3")
x = np.linspace(*ax.get_xlim())
ax.plot(x, x, ls="--", c=".3")
for i in range(len(labels)):
    xy = (measured_flux_list[i]+.1,simulated_flux_list[i]+.1)
    ax.annotate(labels[i],xy)
plt.title(r'R. opacus 13C MFA vs FBA Fluxes', fontSize=20)
plt.xlabel(r'13C MFA flux measurement (mmol/gDW/h)')
plt.ylabel(r'FBA flux prediction (mmol/gDW/h)')
plt.show()

### Simulate with pfba

In [ ]:
pfba_solution = cobra.flux_analysis.pfba(model)
len(pfba_solution.fluxes)

In [ ]:
pfba_flux_dict = {}
measured_reaction_ids = list(measured_glucose_df['Reaction ID'])
for reaction_id in measured_reaction_ids:
    pfba_flux_dict[reaction_id] = pfba_solution.fluxes[reaction_id]

pfba_flux_dict

In [ ]:
measured_flux_list = list(measured_flux_dict.values())
simulated_flux_list = list(pfba_flux_dict.values())
labels = list(measured_flux_dict.keys())

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(measured_flux_list, simulated_flux_list)
x = np.linspace(*ax.get_xlim())
ax.plot(x, x, ls="--", c=".3")
for i in range(len(labels)):
    xy = (measured_flux_list[i]+.1,simulated_flux_list[i]+.1)
    ax.annotate(labels[i],xy)
plt.title(r'R. opacus 13C MFA vs FBA Fluxes', fontSize=20)
plt.xlabel(r'13C MFA flux measurement (mmol/gDW/h)')
plt.ylabel(r'pFBA flux prediction (mmol/gDW/h)')
plt.show()

### Simulate with geometric FBA

In [ ]:
# geometric_fba_sol = cobra.flux_analysis.geometric_fba(model)
# len(geometric_fba_sol.fluxes)

In [ ]:
for r in model.metabolites.get_by_id('mal__L_c').reactions:
    print(r.name, r)